In [0]:
%pip install -q git+https://github.com/stanfordnlp/dspy.git databricks-agents mlflow databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
import mlflow

In [0]:
from mlflow.models.resources import (
    DatabricksVectorSearchIndex,
    DatabricksServingEndpoint,
)
import pkg_resources

# Setup Agent name
uc_model_name = model_name 

# Instantiating Agent
mosaic_agent_rag = RAG(for_mosaic_agent=True)

# Loading optimized DSPy state
mosaic_agent_rag.load("optimized_rag.json")

# Logging Agent into Unity Catalog
with mlflow.start_run(run_name="simple_dspy_rag") as run:
    uc_registered_model_info = mlflow.dspy.log_model(
        mosaic_agent_rag, 
        "model",
        input_example={
            "messages": [
                {
                    "role": "user",
                    "content": "cmd+tab does not work on hidden or minimized windows",
                }
            ]
        },
        task="llm/v1/chat",
        registered_model_name=uc_model_name,
        pip_requirements=[
            "--pre mlflow==2.19.0rc0", 
            f"dspy=={pkg_resources.get_distribution('dspy').version}",
        ],
        resources=[
            DatabricksVectorSearchIndex(
                index_name="main.luis_moros.colbertv2_text_set_index"
            ),
            DatabricksServingEndpoint(
                endpoint_name="neurips_gte_large_en_v1_5_provisioned_throughput"
            ),
            DatabricksServingEndpoint(
                endpoint_name="neurips_llama_v3_2_3b_provisioned_throughput"
            ),
        ],
    )